# Music Recommender System using ALS Algorithm with Apache Spark and Python
+ **Estimated Execution Time (whole script): 2 minutes**
+ **Estimated Time (to complete the project): 8 hours**

## Description

For this project, you are to create a recommender system that will recommend new musical artists to a user based on their listening history. Suggesting different songs or musical artists to a user is important to many music streaming services, such as Pandora and Spotify. In addition, this type of recommender system could also be used as a means of suggesting TV shows or movies to a user (e.g., Netflix). 

To create this system you will be using Spark and the collaborative filtering technique. The instructions for completing this project will be laid out entirely in this file. You will have to implement any missing code as well as answer any questions.

**Submission Instructions:** 
* Add all of your updates to this Jupyter Notebook file and do NOT clear any of the output you get from running your code.
* Upload this file and the genererated HTML onto Moodle as a single zip folder called with your user name.

## Datasets

You will be using some publicly available song data from audioscrobbler, which can be found [here](http://www-etud.iro.umontreal.ca/~bergstrj/audioscrobbler_data.html). However, we modified the original data files so that the code will run in a reasonable time on a single machine. The reduced data files have been suffixed with `_small.txt` and contains only the information relevant to the top 50 most prolific users (highest artist play counts).

The original data file `user_artist_data.txt` contained about 141,000 unique users, and 1.6 million unique artists. About 24.2 million users’ plays of artists are recorded, along with their count.

Note that when plays are scribbled, the client application submits the name of the artist being played. This name could be misspelled or nonstandard, and this may only be detected later. For example, "The Smiths", "Smiths, The", and "the smiths" may appear as distinct artist IDs in the data set, even though they clearly refer to the same artist. So, the data set includes `artist_alias.txt`, which maps artist IDs that are known misspellings or variants to the canonical ID of that artist.

The `artist_data.txt` file then provides a map from the canonical artist ID to the name of the artist.

In [2]:
# Import libraries
import findspark
findspark.init()

from pyspark.mllib.recommendation import *
import random
from operator import *
from collections import defaultdict

In [3]:
# Initialize Spark Context
# YOUR CODE GOES HERE
# Import libraries
from pyspark import SparkContext, SparkConf

sc = SparkContext.getOrCreate()
sc.stop()
sc = SparkContext('local','Recommender')

## Loading data

Load the three datasets into RDDs and name them `artistData`, `artistAlias`, and `userArtistData`. View the README, or the files themselves, to see how this data is formated. Some of the files have tab delimeters while some have space delimiters. Make sure that your `userArtistData` RDD contains only the canonical artist IDs.

In [4]:
# Import test files from location into RDD variables
# YOUR CODE GOES HERE
userArtistData = sc.textFile("data_raw/user_artist_data_small.txt")
artistAlias = sc.textFile("data_raw/artist_alias_small.txt")
artistData = sc.textFile("data_raw/artist_data_small.txt")

## Data Exploration

In the blank below, write some code that with find the users' total play counts. Find the three users with the highest number of total play counts (sum of all counters) and print the user ID, the total play count, and the mean play count (average number of times a user played an artist). Your output should look as follows:
```
User 1059637 has a total play count of 674412 and a mean play count of 1878.
User 2064012 has a total play count of 548427 and a mean play count of 9455.
User 2069337 has a total play count of 393515 and a mean play count of 1519.
```



In [5]:
# Split a sequence into seperate entities and store as int
# YOUR CODE GOES HERE
userArtistDataNew = userArtistData.map(lambda line:line.split(" ")).map(lambda r: (int(r[0]), int(r[1]),int(r[2])))
artistAliasNew = artistAlias.map(lambda line:line.split("\t")).map(lambda r: (int(r[0]), int(r[1])))
artistDataNew = artistData.map(lambda line:line.split("\t")).map(lambda r: (int(r[0]), r[1]))

# Create a dictionary of the 'artistAlias' dataset
# YOUR CODE GOES HERE
dict_art_alias = artistAliasNew.collectAsMap()

# If artistid exists, replace with artistsid from artistAlias, else retain original
# YOUR CODE GOES HERE
dict_keys = list(dict_art_alias.keys())

userArtistDataCorrected = userArtistDataNew.map(lambda x:x if x[1] not in dict_keys else (x[0],dict_art_alias[x[1]],x[2]))

# Create an RDD consisting of 'userid' and 'playcount' objects of original tuple
# YOUR CODE GOES HERE
temp_mean = userArtistDataCorrected.map(lambda r: (int(r[0]), int(r[1]))).groupByKey().map(lambda l:(l[0],len(list(l[1]))))

# Count instances by key and store in broadcast variable
# YOUR CODE GOES HERE
temp_sum = userArtistDataCorrected.map(lambda r: (int(r[0]), int(r[2]))).reduceByKey(lambda a,b:a+b)
# temp_sum = sc.broadcast(temp_sum.collect())
# Compute and display users with the highest playcount along with their mean playcount across artists
# YOUR CODE GOES HERE
temp_three = temp_sum.takeOrdered(3, key=lambda x: -x[1])
# topThree=temp_sum.takeOrdered(3)
temp_three_RDD = sc.parallelize(temp_three)
avg = temp_three_RDD.join(temp_mean).map(lambda x: (x[0],x[1][0],x[1][0]/x[1][1]))
my_list = avg.collect()
from operator import itemgetter
temp_list = sorted(my_list, key=itemgetter(1))
for list_elems in reversed(temp_list):
    print("User "+str(int(list_elems[0]))+" has a total play count of "+str(int(list_elems[1]))+" and a mean play count of "+str(int(list_elems[2])))

User 1059637 has a total play count of 674412 and a mean play count of 1878
User 2064012 has a total play count of 548427 and a mean play count of 9455
User 2069337 has a total play count of 393515 and a mean play count of 1519


####  Splitting Data for Testing

Use the [randomSplit](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.randomSplit) function to divide the data (`userArtistData`) into:
* A training set, `trainData`, that will be used to train the model. This set should constitute 40% of the data.
* A validation set, `validationData`, used to perform parameter tuning. This set should constitute 40% of the data.
* A test set, `testData`, used for a final evaluation of the model. This set should constitute 20% of the data.

Use a random seed value of 13. Since these datasets will be repeatedly used you will probably want to persist them in memory using the [cache](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.cache) function.

In addition, print out the first 3 elements of each set as well as their sizes; if you created these sets correctly, your output should look like the following:
```
[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000114, 2)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000123, 2)]
[(1059637, 1000094, 1), (1059637, 1000112, 423), (1059637, 1000113, 5)]
19761
19862
9858
```

In [6]:
# Split the 'userArtistData' dataset into training, validation and test datasets. Store in cache for frequent access
# YOUR CODE GOES HERE
trainData,validationData,testData=userArtistDataNew.randomSplit([40,40,20],13)
trainData.cache()
validationData.cache()
testData.cache()
# Display the first 3 records of each dataset followed by the total count of records for each datasets
# YOUR CODE GOES HERE
print(trainData.take(3))
print(validationData.take(3))
print(testData.take(3))
print(trainData.count())
print(validationData.count())
print(testData.count())

[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000114, 2)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000123, 2)]
[(1059637, 1000094, 1), (1059637, 1000112, 423), (1059637, 1000113, 5)]
19761
19862
9858


## The Recommender Model

For this project, we will train the model with implicit feedback. You can read more information about this from the collaborative filtering page: [http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html](http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html). The [function you will be using](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS.trainImplicit) has a few tunable parameters that will affect how the model is built. Therefore, to get the best model, we will do a small parameter sweep and choose the model that performs the best on the validation set

Therefore, we must first devise a way to evaluate models. Once we have a method for evaluation, we can run a parameter sweep, evaluate each combination of parameters on the validation data, and choose the optimal set of parameters. The parameters then can be used to make predictions on the test data.

### Model Evaluation

Although there may be several ways to evaluate a model, we will use a simple method here. Suppose we have a model and some dataset of *true* artist plays for a set of users. This model can be used to predict the top X artist recommendations for a user and these recommendations can be compared the artists that the user actually listened to (here, X will be the number of artists in the dataset of *true* artist plays). Then, the fraction of overlap between the top X predictions of the model and the X artists that the user actually listened to can be calculated. This process can be repeated for all users and an average value returned.

For example, suppose a model predicted [1,2,4,8] as the top X=4 artists for a user. Suppose, that user actually listened to the artists [1,3,7,8]. Then, for this user, the model would have a score of 2/4=0.5. To get the overall score, this would be performed for all users, with the average returned.

**NOTE: when using the model to predict the top-X artists for a user, do not include the artists listed with that user in the training data.**

Name your function `modelEval` and have it take a model (the output of ALS.trainImplicit) and a dataset as input. For parameter tuning, the dataset parameter should be set to the validation data (`validationData`). After parameter tuning, the model can be evaluated on the test data (`testData`).

In [7]:
def modelEval(model, dataset):
    
    # All artists in the 'userArtistData' dataset
    # YOUR CODE GOES HERE
    allArtists = userArtistDataCorrected.map(lambda x:x[1]).distinct()
    
    # Set of all users in the current (Validation/Testing) dataset
    # YOUR CODE GOES HERE
    valid_rdd = dataset.map(lambda x: ((x[0]),(x[1]))).groupByKey().map(lambda r: (int(r[0]), list(r[1]))).collectAsMap()
    users_list = list(valid_rdd.keys())
    
    # Create a dictionary of (key, values) for current (Validation/Testing) dataset
    # YOUR CODE GOES HERE
    ## valid_rdd is my dictionary
    
    # Create a dictionary of (key, values) for training dataset
    # YOUR CODE GOES HERE
    train_rdd = trainData.map(lambda x: ((x[0]),(x[1]))).groupByKey().map(lambda r: (int(r[0]), list(r[1]))).collectAsMap()
    
    # For each user, calculate the prediction score i.e. similarity between predicted and actual artists
    # YOUR CODE GOES HERE
    upon = 0
    for user in users_list:
        userPredict = []
        actualArtists = valid_rdd[user]
        ## NOTE: when using the model to predict the top-X artists for a user, do not include the artists listed with that user in the training data.
        artistExcTrain = set(allArtists.collect()) - set(train_rdd[user])
#         artistExcTrain = [x for x in allArtists.collect() if x not in train_rdd[user]]
    # The above method takes more time to work 
    # https://stackoverflow.com/questions/4211209/remove-all-the-elements-that-occur-in-one-list-from-another
    ## Creating a list with (user,artist) pairs
        for art in artistExcTrain:
            userPredict.append((user,art))
        ## Converting list into a RDD        
        userPredictRDD = sc.parallelize(userPredict)
        predictCount = model.predictAll(userPredictRDD)
        predictCountArtist = predictCount.map(lambda l: (l[1],l[2])).takeOrdered(len(actualArtists), key=lambda x: -x[1]) 
        predictCountArtistRDD = sc.parallelize(predictCountArtist)
        predictArtists = predictCountArtistRDD.map(lambda f:f[0]).collect()
        
        a = list(set(predictArtists)&set(actualArtists)) 
        score = len(a)/float(len(predictArtists))
        upon = upon+score
    return (upon/float(len(users_list)))    

### Model Construction

Now we can build the best model possibly using the validation set of data and the `modelEval` function. Although, there are a few parameters we could optimize, for the sake of time, we will just try a few different values for the [rank parameter](http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html#collaborative-filtering) (leave everything else at its default value, **except make `seed`=345**). Loop through the values [2, 10, 20] and figure out which one produces the highest scored based on your model evaluation function.

Note: this procedure may take several minutes to run.

For each rank value, print out the output of the `modelEval` function for that model. Your output should look as follows:
```
The model score for rank 2 is ~0.090431
The model score for rank 10 is ~0.095294
The model score for rank 20 is ~0.090248
```
Step below takes 2 minutes to run. Uncomment to if you wish to run and calculate model score. 

In [8]:
temp_list = []
rankList = [2,10,20]
i = 0
for rank in rankList:
    model = ALS.trainImplicit(trainData, rank , seed=345)
    temp_list.append(modelEval(model,validationData))
print("The model score for rank 2 is ~",round(temp_list[0],6))
print("The model score for rank 10 is ~",round(temp_list[1],6))
print("The model score for rank 20 is ~",round(temp_list[2],6))

The model score for rank 2 is ~ 0.08131
The model score for rank 10 is ~ 0.090155
The model score for rank 20 is ~ 0.083153


Now, using the bestModel, we will check the results over the test data. Your result should be ~`0.0507`.  
Step below takes 1 minute to run. Uncomment last line if you wish to run and calculate model score. 

In [9]:
bestModel = ALS.trainImplicit(trainData, rank=10, seed=345)
round(modelEval(bestModel, testData),6)

0.060424

## Trying Some Artist Recommendations
Using the best model above, predict the top 5 artists for user `1059637` using the [recommendProducts](http://spark.apache.org/docs/1.5.2/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.MatrixFactorizationModel.recommendProducts) function. Map the results (integer IDs) into the real artist name using `artistAlias`. Print the results. The output should look as follows:
```
Artist 0: My Chemical Romance
Artist 1: Something Corporate
Artist 2: Evanescence
Artist 3: Alanis Morissette
Artist 4: Counting Crows
```

In [10]:
# # # Find the top 5 artists for a particular user and list their names
# # # YOUR CODE GOES HERE
rec = bestModel.recommendProducts(1059637,5)
recRDD = sc.parallelize(rec)
recRDDArt = recRDD.map(lambda x:x[1])
art = list(recRDDArt.take(5))
artNameDict = artistDataNew.collectAsMap()
j = 0
for i in art:
    print("Artist "+str(j)+": "+str(artNameDict[i]))
    j=j+1

Artist 0: My Chemical Romance
Artist 1: Something Corporate
Artist 2: Evanescence
Artist 3: Alanis Morissette
Artist 4: Counting Crows
